In [12]:
import numpy as np

from pydrake.math import RigidTransform
from pydrake.multibody.parsing import Parser
from pydrake.systems.analysis import Simulator
from pydrake.all import MultibodyPlant, Frame

from pydrake.solvers import MathematicalProgram, Solve

In [13]:
# setup plant
plant_f = MultibodyPlant(time_step=0.0)
iiwa_url = (
    "package://drake_models/iiwa_description/sdf/iiwa14_no_collision.sdf"
)
(iiwa,) = Parser(plant_f).AddModelsFromUrl(url=iiwa_url)

W = plant_f.world_frame()
L0 = plant_f.GetFrameByName("iiwa_link_0", iiwa)
L7 = plant_f.GetFrameByName("iiwa_link_7", iiwa)

plant_f.WeldFrames(W, L0)
plant_f.Finalize()

In [14]:
# create context and autodiff plant
context_f = plant_f.CreateDefaultContext()
plant_ad = plant_f.ToAutoDiffXd()
context_ad = plant_ad.CreateDefaultContext()

def resolve_frame(plant: MultibodyPlant, F: Frame):
    return plant.GetFrameByName(F.name(), F.model_instance())

p_WT = [0.1, 0.1, 0.6]
def link_7_distance_to_target(q):
    if q.dtype == float:
        plant = plant_f
        context = context_f
    else:
        plant = plant_ad
        context = context_ad
    plant.SetPositions(context, iiwa, q)
    X_WL7 = plant.CalcRelativeTransform(context, resolve_frame(plant, W), resolve_frame(plant, L7))
    p_TL7 = X_WL7.translation() - p_WT
    return p_TL7.dot(p_TL7) #returns squared distance

link_7_distance_to_target_vector = lambda q: [link_7_distance_to_target(q)]

In [15]:
prog = MathematicalProgram()
q = prog.NewContinuousVariables(plant_f.num_positions())
q0 = np.zeros(plant_f.num_positions())

prog.AddCost((q-q0).dot(q-q0)) # add quadratic cost

prog.AddConstraint(
    link_7_distance_to_target_vector, lb=[0.1], ub=[0.2], vars=q
)

In [16]:
result = Solve(prog, initial_guess=q0)
print(f"Success? {result.is_success()}")
print(result.get_solution_result())
q_sol = result.GetSolution(q)
print(q_sol)

print(f"Initial distance: {link_7_distance_to_target(q0):.3f}")
print(f"Solution distance: {link_7_distance_to_target(q_sol):.3f}")

Success? True
SolutionResult.kSolutionFound
[ 0.18293302 -0.15345868  0.22224834 -1.24010409  0.01028776  0.28116727
  0.        ]
Initial distance: 0.457
Solution distance: 0.200


# Alternative Formulation

In [17]:
prog = MathematicalProgram()

q = prog.NewContinuousVariables(plant_f.num_positions())
# Define nominal configuration.
q0 = np.zeros(plant_f.num_positions())

# Add custom cost.
prog.AddCost(link_7_distance_to_target, vars=q)

In [ ]:
result = Solve(prog, initial_guess=q0)

print(f"Success? {result.is_success()}")
print(result.get_solution_result())
q_sol = result.GetSolution(q)
print(q_sol)

print(f"Initial distance: {link_7_distance_to_target(q0):.3f}")
print(f"Solution distance: {link_7_distance_to_target(q_sol):.3f}")